<a href="https://colab.research.google.com/github/RejwankabirHamim/Machine-Learning-Sessional-/blob/main/Linear%20Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi-variable Linear Regression

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [5]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/property_listing_data_in_Bangladesh.csv')

In [ ]:
data.head()

,title,beds,bath,area,adress,type,purpose,flooPlan,url,lastUpdated,price
0,Eminent Apartment Of 2200 Sq Ft Is Vacant For ...,3,4,"2,200 sqft","Block A, Bashundhara R-A, Dhaka",Apartment,For Rent,https://images-cdn.bproperty.com/thumbnails/10...,https://www.bproperty.com/en/property/details-...,"August 13, 2022",50 Thousand
1,"Apartment Ready To Rent In South Khulshi, Near...",3,4,"1,400 sqft","South Khulshi, Khulshi, Chattogram",Apartment,For Rent,https://images-cdn.bproperty.com/thumbnails/44...,https://www.bproperty.com/en/property/details-...,"January 25, 2022",30 Thousand
2,"Smartly priced 1950 SQ FT apartment, that you ...",3,4,"1,950 sqft","Block F, Bashundhara R-A, Dhaka",Apartment,For Rent,https://images-cdn.bproperty.com/thumbnails/11...,https://www.bproperty.com/en/property/details-...,"February 22, 2023",30 Thousand
3,2000 Sq Ft Residential Apartment Is Up For Ren...,3,3,"2,000 sqft","Sector 9, Uttara, Dhaka",Apartment,For Rent,https://images-cdn.bproperty.com/thumbnails/14...,https://www.bproperty.com/en/property/details-...,"October 28, 2021",35 Thousand
4,Strongly Structured This 1650 Sq. ft Apartment...,3,4,"1,650 sqft","Block I, Bashundhara R-A, Dhaka",Apartment,For Rent,https://images-cdn.bproperty.com/thumbnails/10...,https://www.bproperty.com/en/property/details-...,"February 19, 2023",25 Thousand


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7557 entries, 0 to 7556
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        7557 non-null   object
 1   beds         7557 non-null   object
 2   bath         7557 non-null   object
 3   area         7557 non-null   object
 4   adress       7557 non-null   object
 5   type         7557 non-null   object
 6   purpose      7557 non-null   object
 7   flooPlan     7556 non-null   object
 8   url          7557 non-null   object
 9   lastUpdated  7557 non-null   object
 10  price        7557 non-null   object
dtypes: object(11)
memory usage: 649.6+ KB


In [ ]:
data.drop_duplicates(inplace=True)
data.drop(data[data['type'] == 'Duplex'].index, inplace = True)
data.drop(data[data['type'] == 'Building'].index, inplace = True)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7489 entries, 0 to 7556
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        7489 non-null   object
 1   beds         7489 non-null   object
 2   bath         7489 non-null   object
 3   area         7489 non-null   object
 4   adress       7489 non-null   object
 5   type         7489 non-null   object
 6   purpose      7489 non-null   object
 7   flooPlan     7488 non-null   object
 8   url          7489 non-null   object
 9   lastUpdated  7489 non-null   object
 10  price        7489 non-null   object
dtypes: object(11)
memory usage: 702.1+ KB


In [ ]:
data.dropna(inplace=True)

In [ ]:
#data.drop(['flooPlan'], axis=1)
data.drop(data.columns[[0, 4, 5,6,7,8,9]], axis=1, inplace=True)
data.head()

,beds,bath,area,price
0,3,4,"2,200 sqft",50 Thousand
1,3,4,"1,400 sqft",30 Thousand
2,3,4,"1,950 sqft",30 Thousand
3,3,3,"2,000 sqft",35 Thousand
4,3,4,"1,650 sqft",25 Thousand


In [ ]:
data['beds']=data['beds'].replace(regex=[r'\D+'], value="")
data['bath']=data['bath'].replace(regex=[r'\D+'], value="")
data['area']=data['area'].replace(regex=[r'\D+'], value="")
data.head()

,beds,bath,area,price
0,3,4,2200,50 Thousand
1,3,4,1400,30 Thousand
2,3,4,1950,30 Thousand
3,3,3,2000,35 Thousand
4,3,4,1650,25 Thousand


In [ ]:
def calc(prev):
    prev = str(prev)
    prev = prev.lower()
    if 'thousand' in prev:
        prev = re.sub(r'[^\d.]+', '', prev)
        prev = float(prev) * 1000.0
    elif 'lakh' in prev:
        prev = re.sub(r'[^\d.]+', '', prev)
        prev = float(prev) * 100000.0
    # try:
    #     entry = float(entry)
    # except ValueError:
    #     pass
    return prev

data['price'] = data['price'].apply(calc)

#data.head()

In [ ]:
data.head()

,beds,bath,area,price
0,3,4,2200,50000.0
1,3,4,1400,30000.0
2,3,4,1950,30000.0
3,3,3,2000,35000.0
4,3,4,1650,25000.0


In [ ]:
data['area'] = data['area'].astype(float)
data['beds'] = data['beds'].astype(int)
data['bath'] = data['bath'].astype(int)

In [ ]:

from scipy.stats import zscore
data['beds'] = zscore(data['beds'])
data['bath'] = zscore(data['bath'])
data['area'] = zscore(data['area'])
data['price'] = zscore(data['price'])
# Display the updated DataFrame
data.head()

,beds,bath,area,price
0,0.448143,1.615050,1.348291,0.441386
1,0.448143,1.615050,0.115552,-0.024108
2,0.448143,1.615050,0.963060,-0.024108
3,0.448143,0.405942,1.040106,0.092265
4,0.448143,1.615050,0.500783,-0.140482


In [ ]:
def compute_cost(X, y, w, b):

    m = X.shape[0]
    cost = 0.0
    for i in range(m):
        f_wb_i = np.dot(X[i], w) + b
        cost = cost + (f_wb_i - y[i])**2
    cost = cost / (2 * m)
    return cost

In [ ]:

def compute_gradient(X, y, w, b):

    m,n = X.shape
    dj_dw = np.zeros((n,))
    dj_db = 0.

    for i in range(m):
        err = (np.dot(X[i], w) + b) - y[i]
        for j in range(n):
            dj_dw[j] = dj_dw[j] + err * X[i, j]
        dj_db = dj_db + err
    dj_dw = dj_dw / m
    dj_db = dj_db / m

    return dj_db, dj_dw

In [ ]:
def gradient_descent(X, y,X_val,y_val, w_in, b_in, cost_function, gradient_function, alpha, num_iters):
    history = {'train_loss': [], 'val_loss': []}
    w = w_in
    b = b_in

    for i in range(num_iters):
        dj_db,dj_dw = gradient_function(X, y, w, b)

        w = w - alpha * dj_dw
        b = b - alpha * dj_db

        if i<100000:
            history['train_loss'].append( cost_function(X, y, w, b))
            history['val_loss'].append( cost_function(X_val, y_val, w, b))


        print(f"Epoch {i + 1}/{num_iters} - Training Loss: {history['train_loss'][-1]:8.2f}, Validation Loss: {history['val_loss'][-1]:8.2f}")
       # print(f"Iteration {i:4d}: Cost {J_history[-1]:8.2f}   ")

    return w, b, history

In [ ]:
from sklearn.model_selection import train_test_split
X = data[['beds', 'bath', 'area']].values
y = data['price'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25)

In [ ]:

initial_w = np.zeros(X_train.shape[1])
initial_b = 0.
iterations = 1000
alpha = 5.0e-2
w_final, b_final, J_hist = gradient_descent(X_train, y_train,X_val,y_val, initial_w, initial_b,compute_cost, compute_gradient, alpha,iterations)
print(f"b,w found by gradient descent: {b_final:0.2f},{w_final} ")


Epoch 1/1000 - Training Loss:     0.43, Validation Loss:     0.34
Epoch 2/1000 - Training Loss:     0.39, Validation Loss:     0.31
Epoch 3/1000 - Training Loss:     0.36, Validation Loss:     0.28
Epoch 4/1000 - Training Loss:     0.34, Validation Loss:     0.26
Epoch 5/1000 - Training Loss:     0.32, Validation Loss:     0.25
Epoch 6/1000 - Training Loss:     0.31, Validation Loss:     0.23
Epoch 7/1000 - Training Loss:     0.29, Validation Loss:     0.22
Epoch 8/1000 - Training Loss:     0.28, Validation Loss:     0.22
Epoch 9/1000 - Training Loss:     0.27, Validation Loss:     0.21
Epoch 10/1000 - Training Loss:     0.27, Validation Loss:     0.20
Epoch 11/1000 - Training Loss:     0.26, Validation Loss:     0.20
Epoch 12/1000 - Training Loss:     0.25, Validation Loss:     0.20
Epoch 13/1000 - Training Loss:     0.25, Validation Loss:     0.19
Epoch 14/1000 - Training Loss:     0.25, Validation Loss:     0.19
Epoch 15/1000 - Training Loss:     0.24, Validation Loss:     0.19
Epoc

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
m,_ = X_test.shape
print(f"evaluating the model on the test dataset.")
test_loss = compute_cost(X_test, y_test, w_final,b_final)
print(f"Test Loss: {test_loss}")
for i in range(m):
    print(f"prediction: {np.dot(X_test[i], w_final) + b_final:0.2f}, target value: {y_test[i]}")
regressor = LinearRegression()
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse}")

# Calculate R-squared score
r2 = r2_score(y_test, y_pred)
print(f"R-squared Score: {r2}")

evaluating the model on the test dataset.
Test Loss: 0.27081109627594174
prediction: -0.25, target value: -0.23358061346635126
prediction: -0.71, target value: -0.4197781946338084
prediction: -0.38, target value: -0.2568553111122834
prediction: -0.54, target value: -0.47796493874863877
prediction: 0.09, target value: -0.37322879934194414
prediction: -0.67, target value: -0.37322879934194414
prediction: -0.34, target value: -0.4663275899256727
prediction: -0.38, target value: -0.37322879934194414
prediction: -0.58, target value: -0.3965034969878763
prediction: -0.13, target value: 0.44138561826568096
prediction: -0.42, target value: -0.2568553111122834
prediction: -0.16, target value: -0.2568553111122834
prediction: 0.29, target value: 0.09226515357669877
prediction: -0.07, target value: -0.2940948273457748
prediction: -0.29, target value: -0.37322879934194414
prediction: 0.12, target value: -0.14048182288262268
prediction: -0.38, target value: -0.4546902411027066
prediction: -0.58, tar